<a href="https://colab.research.google.com/github/Berlingrad/MLinCyberSecurityProject/blob/XuDevelopment/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#DNN Back Door Detector

## Preprocessing

###Data Download

In [1]:
!git clone https://github.com/csaw-hackml/CSAW-HackML-2020.git

!gdown https://drive.google.com/uc?id=1P8PTL62x3cfpV9mrC0unqZjRFhlTTOSG
!gdown https://drive.google.com/uc?id=19OKCkY2CjV3ASkOe6nMSYTsOVcxAoCnA
!gdown https://drive.google.com/uc?id=1XtYnM-IopU-QYVc99U51EiDvI5zxK0nV

!mv /content/*.h5 /content/CSAW-HackML-2020/data

##!wget https://raw.githubusercontent.com/bolunwang/backdoor/master/visualizer.py

fatal: destination path 'CSAW-HackML-2020' already exists and is not an empty directory.
Downloading...
From: https://drive.google.com/uc?id=1P8PTL62x3cfpV9mrC0unqZjRFhlTTOSG
To: /content/sunglasses_poisoned_data.h5
398MB [00:03, 123MB/s]
Downloading...
From: https://drive.google.com/uc?id=19OKCkY2CjV3ASkOe6nMSYTsOVcxAoCnA
To: /content/clean_validation_data.h5
716MB [00:03, 222MB/s]
Downloading...
From: https://drive.google.com/uc?id=1XtYnM-IopU-QYVc99U51EiDvI5zxK0nV
To: /content/clean_test_data.h5
398MB [00:07, 55.1MB/s]


In [2]:
import os
import gdown
import numpy as np
import random
import h5py
import glob2
import tempfile

import keras
import tensorflow as tf
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
!pip install -q tensorflow-model-optimization
import tensorflow_model_optimization as tfmot

if not tf.config.experimental.list_physical_devices('GPU'):
    raise Exception("Please change runtime type to GPU")


In [3]:
def load_dataset(data_filename, keys=None):
    dataset = {}
    with h5py.File(data_filename, 'r') as hf:
        if keys is None:
            for name in hf:
                dataset[name] = np.array(hf.get(name))
        else:
            for name in keys:
                dataset[name] = np.array(hf.get(name))

    return dataset

def load_models(model_dir):
    models = []
    for net in glob2.glob("/content/CSAW-HackML-2020/models/*net.h5"):
        weights = glob2.glob(net.split("_")[0]+"*weights.h5")[0]
       
        m = load_model(net)
        m.load_weights(weights)

        models.append(m)
    return models

###Load Data

In [4]:
SG_poisoned_path = "/content/CSAW-HackML-2020/data/sunglasses_poisoned_data.h5"
SG_poisoned = load_dataset(SG_poisoned_path)

clean_validation_path = "/content/CSAW-HackML-2020/data/clean_validation_data.h5"
valid_clean = load_dataset(clean_validation_path)

clean_test_path = "/content/CSAW-HackML-2020/data/clean_test_data.h5"
test_clean = load_dataset(clean_test_path)

SG_poisoned['data'] = SG_poisoned['data'].transpose((0,2,3,1))
valid_clean['data'] = valid_clean['data'].transpose((0,2,3,1))
test_clean['data'] = test_clean['data'].transpose((0,2,3,1))

 
 ##Vectorize label array to fit the model output
outLen = 1283 ##Length of output array
ySG = np.zeros((SG_poisoned['label'].shape[0], outLen))
yValid = np.zeros((valid_clean['label'].shape[0], outLen))
yTest = np.zeros((test_clean['label'].shape[0], outLen))

print(valid_clean['data'].shape)
print(yValid.shape)

for i in range(SG_poisoned['label'].shape[0]):
    ySG[i][SG_poisoned['label'][i]] = 1

for i in range(valid_clean['label'].shape[0]):
    yValid[i][int(valid_clean['label'][i])] = 1

for i in range(test_clean['label'].shape[0]):
    yTest[i][test_clean['label'][i]] = 1



(11547, 55, 47, 3)
(11547, 1283)


###Data Parameters

In [5]:
##TODO

##SG_nClass = 
##vc_nClass = 

###Load Models

In [6]:

model_dir = "/content/CSAW-HackML-2020/models"

models = load_models(model_dir)


In [7]:
out = models[0].predict(valid_clean['data'])
print(np.argmax(out[0]))
print(out.shape)
print(yValid.shape)

43
(11547, 1283)
(11547, 1283)


### Model Summary

In [8]:
##TODO
for i, model in enumerate(models):
    print("model ", i, ":")
    model.summary()

     

model  0 :
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 55, 47, 3)]  0                                            
__________________________________________________________________________________________________
conv_1 (Conv2D)                 (None, 52, 44, 20)   980         input[0][0]                      
__________________________________________________________________________________________________
pool_1 (MaxPooling2D)           (None, 26, 22, 20)   0           conv_1[0][0]                     
__________________________________________________________________________________________________
conv_2 (Conv2D)                 (None, 24, 20, 40)   7240        pool_1[0][0]                     
_________________________________________________________________________________

##Pruning

In [9]:
##TODO
pruned_models = []
for i, base_model in enumerate(models):
    model_for_pruning = tfmot.sparsity.keras.prune_low_magnitude(base_model)

    log_dir = tempfile.mkdtemp()
    callbacks = [
        tfmot.sparsity.keras.UpdatePruningStep(),
        tfmot.sparsity.keras.PruningSummaries(log_dir=log_dir)
    ]
    model_for_pruning.compile(
      loss=tf.keras.losses.categorical_crossentropy,
      optimizer='adam',
      metrics=['accuracy']
    )
    model_for_pruning.fit(test_clean['data'], yTest,
                          callbacks=callbacks,
                          epochs=5
                          )

    pruned_models.append(model_for_pruning)
 

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/base_layer.py:2281: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


Epoch 1/5
401/401 [==============================] - 10s 17ms/step - loss: 143.7407 - accuracy: 0.7884
Epoch 2/5
401/401 [==============================] - 6s 16ms/step - loss: 12.4424 - accuracy: 0.9022
Epoch 3/5
401/401 [==============================] - 6s 16ms/step - loss: 4.7603 - accuracy: 0.9391
Epoch 4/5
401/401 [==============================] - 6s 16ms/step - loss: 2.1762 - accuracy: 0.9657
Epoch 5/5
401/401 [==============================] - 6s 16ms/step - loss: 1.6167 - accuracy: 0.9671
Epoch 1/5
401/401 [==============================] - 9s 17ms/step - loss: 260.9475 - accuracy: 0.6975
Epoch 2/5
401/401 [==============================] - 6s 16ms/step - loss: 12.1864 - accuracy: 0.8891
Epoch 3/5
401/401 [==============================] - 6s 16ms/step - loss: 2.6483 - accuracy: 0.9445
Epoch 4/5
401/401 [==============================] - 6s 16ms/step - loss: 1.0439 - accuracy: 0.9684
Epoch 5/5
401/401 [==============================] - 6s 16ms/step - loss: 0.4452 - accuracy: 

In [10]:
for i, model in enumerate(pruned_models):
    print("model ", i, ":")
    model.summary()


model  0 :
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 55, 47, 3)]  0                                            
__________________________________________________________________________________________________
prune_low_magnitude_conv_1 (Pru (None, 52, 44, 20)   1942        input[0][0]                      
__________________________________________________________________________________________________
prune_low_magnitude_pool_1 (Pru (None, 26, 22, 20)   1           prune_low_magnitude_conv_1[0][0] 
__________________________________________________________________________________________________
prune_low_magnitude_conv_2 (Pru (None, 24, 20, 40)   14442       prune_low_magnitude_pool_1[0][0] 
_________________________________________________________________________________

##Fine-tuning

In [11]:
for i, m in enumerate(pruned_models):
    m.compile(optimizer=keras.optimizers.Adam(1e-6),
              loss=tf.keras.losses.categorical_crossentropy,
              metrics=['accuracy']
              )
    model_for_pruning.fit(test_clean['data'], yTest,
                          epochs=10
                          )

Epoch 1/10
401/401 [==============================] - 5s 12ms/step - loss: 2.9346 - accuracy: 0.9491
Epoch 2/10
401/401 [==============================] - 5s 12ms/step - loss: 1.8096 - accuracy: 0.9617
Epoch 3/10
401/401 [==============================] - 5s 12ms/step - loss: 1.8066 - accuracy: 0.9637
Epoch 4/10
401/401 [==============================] - 5s 12ms/step - loss: 1.4160 - accuracy: 0.9670
Epoch 5/10
401/401 [==============================] - 5s 12ms/step - loss: 1.4274 - accuracy: 0.9627
Epoch 6/10
401/401 [==============================] - 5s 12ms/step - loss: 1.2406 - accuracy: 0.9620
Epoch 7/10
401/401 [==============================] - 5s 12ms/step - loss: 0.9613 - accuracy: 0.9663
Epoch 8/10
401/401 [==============================] - 5s 12ms/step - loss: 0.8585 - accuracy: 0.9684
Epoch 9/10
401/401 [==============================] - 5s 12ms/step - loss: 0.8159 - accuracy: 0.9683
Epoch 10/10
401/401 [==============================] - 5s 12ms/step - loss: 0.6872 - accura

##Validation

In [12]:
##Intermediate quick evalution using built-in Keras functions and the poisoned set

for i, m in enumerate(pruned_models):
    print("model ", i, "Accurcy on clean validation set: ")
    m.evaluate(valid_clean['data'], yValid) 
    print("model ", i, "Accurcy on poisoned set: ")
    m.evaluate(SG_poisoned['data'], ySG) 

model  0 Accurcy on clean validation set: 
361/361 [==============================] - 2s 4ms/step - loss: 8.7659 - accuracy: 0.9093
model  0 Accurcy on poisoned set: 
401/401 [==============================] - 2s 4ms/step - loss: 2026.4587 - accuracy: 0.0000e+00
model  1 Accurcy on clean validation set: 
361/361 [==============================] - 2s 4ms/step - loss: 5.4992 - accuracy: 0.8847
model  1 Accurcy on poisoned set: 
401/401 [==============================] - 2s 4ms/step - loss: 30.0270 - accuracy: 0.5190
model  2 Accurcy on clean validation set: 
361/361 [==============================] - 2s 4ms/step - loss: 2.6118 - accuracy: 0.8869
model  2 Accurcy on poisoned set: 
401/401 [==============================] - 2s 4ms/step - loss: 445.1712 - accuracy: 0.0000e+00
